# Test notebook

This notebook can be used to test a besca version to ensure that everything is still functional.

It shows some analysis results presentations and should be checked before a release.

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

In [ ]:
import sys
sys.path1 = [x for x in sys.path if "besca2.2" in x]
sys.path2 = [x for x in sys.path if "besca2.2" not in x]
sys.path = sys.path1 + sys.path2

In [ ]:
import scanpy as sc
import besca as bc
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

vers = bc.print_software_versions()

# testing of helper functions

In [ ]:
adata = bc.datasets.pbmc3k_processed()
adata_raw = bc.get_raw(adata)

# testing of count functions

In [ ]:
adata = bc.datasets.Baron2016_processed()

In [ ]:
adata

In [ ]:
bc.tl.count_occurrence_subset(adata=adata, count_variable='leiden', subset_variable='Individual')

In [ ]:
bc.tl.count_occurrence_subset_conditions(adata, subset_variable = 'dblabel', count_variable = 'leiden', condition_identifier = 'Individual', )

In [ ]:
fig = bc.pl.celllabel_quant_boxplot(adata, count_variable = 'leiden', subset_variable = 'dblabel', condition_identifier = 'Individual',  plot_percentage = True);

In [ ]:
fig = bc.pl.celllabel_quant_stackedbar(adata, count_variable = 'leiden', subset_variable = 'Individual');

# Testing with PBMC3k Dataset

In [ ]:
adata = bc.datasets.pbmc3k_raw()

In [ ]:
#define thresholds
min_genes = 600
min_cells = 2
min_UMI = 600
max_UMI = 6500
max_mito = 0.05
max_genes = 1900

#define outdir
outdir = '../besca_test/pbmc3k/'

#set randomseed
random_seed = 0

In [ ]:
#visualize filtering thresholds
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6))= plt.subplots(ncols=3, nrows=2)
fig.set_figwidth(15)
fig.set_figheight(8)
fig.tight_layout(pad=4.5)

bc.pl.kp_genes(adata, min_genes=min_genes, ax = ax1)
bc.pl.kp_cells(adata, min_cells=min_cells, ax = ax2)
bc.pl.kp_counts(adata, min_counts=min_UMI, ax = ax3)
bc.pl.max_counts(adata, max_counts=max_UMI, ax = ax4)
bc.pl.max_mito(adata, max_mito=max_mito, annotation_type='SYMBOL', species='human', ax = ax5)
bc.pl.max_genes(adata, max_genes=max_genes)

In [ ]:
#perform filtering of the thresholds
adata = bc.pp.filter(adata, max_counts=max_UMI, max_genes=max_genes, max_mito=max_mito,min_genes=min_genes, min_counts=min_UMI, min_cells=min_cells)

In [ ]:
sc.pl.violin(adata, ['n_counts', 'n_genes', 'percent_mito'], multi_panel=True, jitter = 0.4)

In [ ]:
#normalize our data (not an internal besca function)
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4) 

#log transform normalized UMI-counts (+1 offset) and store as new "raw" data
adata.raw = sc.pp.log1p(adata, copy=True)

In [ ]:
#export our data
bc.export.X_to_mtx(adata=adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME','normalized_counts', 'cp10k'), write_metadata=True, geneannotation='SYMBOL', additional_geneannotation='ENSEMBL')

In [ ]:
#identify genes with variable expression
filter_result = sc.pp.filter_genes_dispersion(adata.X, min_mean = 0.0125, max_mean=5, min_disp = 0.5) 
sc.pl.filter_genes_dispersion(filter_result)
nbr_variable_genes = sum(filter_result.gene_subset)
print('number of variable genes selected ', nbr_variable_genes )

#apply filter on data
adata = adata[:, filter_result.gene_subset]

#log transform our data
sc.pp.log1p(adata)

#regress-out
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

#scale data
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, random_state=random_seed, svd_solver='arpack')

sc.pp.neighbors(adata, n_neighbors=10, random_state = random_seed)

sc.tl.umap(adata, random_state = random_seed)

sc.tl.leiden(adata, random_state = random_seed)

#plot clusters
sc.settings.set_figure_params(dpi=90)
sc.pl.umap(adata, color=['leiden'], projection='2d', edgecolor = 'none')

#also perform TSNE
sc.tl.tsne(adata)
sc.pl.tsne(adata, color = ['leiden'])

In [ ]:
#write out regressed counts
bc.export.X_to_mtx(adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME', 'normalized_counts', 'regressedOut'), geneannotation='SYMBOL', write_metadata= True, additional_geneannotation='ENSEMBL')

In [ ]:
#export values saved in .raw
#this is an example of how to use this function, in this case we don't need it 
#bc.export.raw_to_mtx(adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME', 'normalized_counts', 'regressedOut_raw'), geneannotation='SYMBOL', write_metadata= True, additional_geneannotation='ENSEMBL')

In [ ]:
bc.export.clustering(adata, outpath = os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'))
bc.export.labeling_info(outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'))

In [ ]:
bc.export.analysis_metadata(adata, outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME'), n_pcs= 3, umap=True, tsne=True)

In [ ]:
#marker gene analysis
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', use_raw = True, n_genes = adata.raw.X.shape[1])

In [ ]:
#export rank files
bc.export.ranked_genes(adata=adata, outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'), type='wilcox')

In [ ]:
#plot markers documented in Seurat tutorial to label celltypes
sc.pl.umap(adata=adata, color = ['leiden'], edgecolor = 'none')

In [ ]:
new_labels = ["mixed", #0
              "mixed", #1
              "CD14+ monocyte", #2
              "mixed", #3
              "B-cell", #4
              "FCGR3A+ monocyte", #5
              "mixed", #6
              "pDC"] #7

bc.tl.annotate_cells_clustering(adata, new_labels, clustering_label = 'leiden')

In [ ]:
#preserve these less well defined labels for some plotting examples later on
adata.obs['highlevel_celltype'] = adata.obs.get('celltype').tolist()

In [ ]:

#bc.tl.sig.combined_signature_score(adata=adata, GMT_file= '../besca/datasets/genesets/Immune.gmt', use_raw=True, verbose = False)

In [ ]:
#sc.pl.umap(adata, color= [col for col in adata.obs.columns if 'Bcell'  in col],
#      color_map='viridis', projection='2d'
#            )

# Demonstrate additional BESCA features

## reclustering and exporting new celltype annotations:

you can choose clusters you wish to subcluster, here in the example 0, 1, 3, 6 since they can't easily be held appart

In [ ]:
adata_subset = bc.tl.rc.recluster(adata, celltype=('0', '1', '3', '6'), celltype_label = 'leiden',
                                  resolution = 1.3, method='leiden')

In [ ]:
sc.pl.umap(adata_subset, color = ['leiden', 'CD3G', 'CD8A', 'CD4', 'IL7R', 'NKG7', 'GNLY'])

In [ ]:
adata.obs.head()

In [ ]:
new_labels = ["CD4 T-cell", #0
              "CD4 T-cell", #1
              "NK cell", #2
              "CD8 T-cell", #3
              "CD8 T-cell", #4
              "CD8 T-cell", #5
              "CD4 T-cell",#6
              "CD8 T-cell", #7
              "CD4 T-cell", #8
              "CD4 T-cell", #9
              "CD4 T-cell", #10
              "CD4 T-cell",#11
              "CD4 T-cell",#12
              "CD4 T-cell", #13
             ]

## for demonstration purposes, we make sure that the new labels are always
## the same long as the old labels
old_labels = adata_subset.obs.get("leiden").value_counts().index.tolist()
if len(new_labels)>len(old_labels):
    new_labels = new_labels[:len(old_labels)]
elif len(new_labels)<len(old_labels):
    new_labels.extend(["CD4 T-cell"]*(len(old_labels)-len(new_labels)))

In [ ]:
bc.tl.rc.annotate_new_cellnames(adata, adata_subset, names=new_labels, method='leiden')

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.celltype.value_counts()

In [ ]:
#export celltypes
bc.export.labeling(adata=adata, outpath = os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'celltype'), column='celltype')
bc.export.labeling_info(outpath = os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'celltype'), method='manual celltype annotation based on marker expression', annotated_version_of='leiden', expert=True,default=False, public=False, reference=True, description='manual celltype annotation based on the expression of marker genes')

## Adding an already done labeling into your adata object

In [ ]:
#note many of the entries in celltype will be named not labeled since they were filtered out before the proper labeling was performed
adata = bc.datasets.pbmc3k_raw()
adata.obs.head()

In [ ]:
bc.Import.add_cell_labeling(adata, os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'celltype'), label='celltype')

In [ ]:
adata.obs.head()

# Testing with PMBC raw dataset

In [ ]:
adata = bc.datasets.pbmc3k_raw()

In [ ]:
#define thresholds
min_genes = 600
min_cells = 2
min_UMI = 1600
max_UMI = 15000
max_mito = 0.15
max_genes = 3000

#define outdir
outdir = '../besca_test/pbmc_storage/'
#set randomseed
random_seed = 0

In [ ]:
#visualize filtering thresholds
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6))= plt.subplots(ncols=3, nrows=2)
fig.set_figwidth(15)
fig.set_figheight(8)
fig.tight_layout(pad=4.5)

bc.pl.kp_genes(adata, min_genes=min_genes, ax = ax1)
bc.pl.kp_cells(adata, min_cells=min_cells, ax = ax2)
bc.pl.kp_counts(adata, min_counts=min_UMI, ax = ax3)
bc.pl.max_counts(adata, max_counts=max_UMI, ax = ax4)
bc.pl.max_mito(adata, max_mito=max_mito, annotation_type='SYMBOL', species='human', ax = ax5)
bc.pl.max_genes(adata, max_genes=max_genes)

In [ ]:
#perform filtering of the thresholds
adata = bc.pp.filter(adata, max_counts=max_UMI, max_genes=max_genes, max_mito=max_mito,min_genes=min_genes, min_counts=min_UMI, min_cells=min_cells)

In [ ]:
sc.pl.violin(adata, ['n_counts', 'n_genes', 'percent_mito'], multi_panel=True, jitter = 0.4)

In [ ]:
#normalize our data (not an internal besca function)
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4) 

#log transform normalized UMI-counts (+1 offset) and store as new "raw" data
adata.raw = sc.pp.log1p(adata, copy=True)

In [ ]:
#export our data
bc.export.X_to_mtx(adata=adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME','normalized_counts', 'cp10k'), write_metadata=True, geneannotation='SYMBOL', additional_geneannotation='ENSEMBL')

In [ ]:
#identify genes with variable expression
filter_result = sc.pp.filter_genes_dispersion(adata.X, min_mean = 0.0125, max_mean=5, min_disp = 0.5) 
sc.pl.filter_genes_dispersion(filter_result)
nbr_variable_genes = sum(filter_result.gene_subset)
print('number of variable genes selected ', nbr_variable_genes )

#apply filter on data
adata = adata[:, filter_result.gene_subset]

#log transform our data
sc.pp.log1p(adata)

#regress-out
sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

#scale data
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata, random_state=random_seed, svd_solver='arpack')

sc.pp.neighbors(adata, n_neighbors=10, random_state = random_seed)

sc.tl.umap(adata, random_state = random_seed)

sc.tl.leiden(adata, random_state = random_seed)

#plot clusters
sc.settings.set_figure_params(dpi=90)
sc.pl.umap(adata, color=['leiden'], projection='2d', edgecolor = 'none')

#also perform TSNE
sc.tl.tsne(adata)
sc.pl.tsne(adata, color = ['leiden'])

In [ ]:
#write out regressed counts
# THIS IS SLOW
bc.export.X_to_mtx(adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME', 'normalized_counts', 'regressedOut'), geneannotation='SYMBOL', write_metadata= True, additional_geneannotation='ENSEMBL')

In [ ]:
#export values saved in .raw
#this is an example of how to use this function, in this case we don't need it 
#bc.export.raw_to_mtx(adata, outpath=os.path.join(outdir, 'analyzed', 'ANALYSIS_NAME', 'normalized_counts', 'regressedOut_raw'), geneannotation='SYMBOL', write_metadata= True, additional_geneannotation='ENSEMBL')

In [ ]:
bc.export.clustering(adata, outpath = os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'))
bc.export.labeling_info(outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'))

In [ ]:
bc.export.analysis_metadata(adata, outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME'), n_pcs= 3, umap=True, tsne=True)

In [ ]:
#marker gene analysis
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', use_raw = True, n_genes = adata.raw.X.shape[1])

In [ ]:
#export rank files
bc.export.ranked_genes(adata=adata, outpath=os.path.join(outdir,'analyzed', 'ANALYSIS_NAME', 'labelings', 'leiden'), type='wilcox')

In [ ]:
#plot markers documented in Seurat tutorial to label celltypes
sc.pl.umap(adata=adata, color = ['leiden'], edgecolor = 'none')

## Counting occurrences of labels and have matching colors with the UMAP.

In [ ]:
#generate a count table of the chosen count_variable
counts = bc.tl.count_occurrence(adata=adata, count_variable='leiden', add_percentage=True)
display(counts)

In [ ]:
ncolors = ( counts.shape)[0]

In [ ]:
import seaborn as sns
#generate a basic bar plot of the table above with customed palette
current_palette ={str(i): sns.color_palette('colorblind', ncolors)[i] for i in range(ncolors)}
fig = plt.figure(figsize=(15, 6))

ax1 = fig.add_subplot(1, 2, 1)
ax1 = sns.barplot(data=counts, x=counts.index.tolist(), y='Counts', palette = current_palette)
plt.xticks(rotation=90)

ax2 = fig.add_subplot(1, 2, 2)
ax2 = sns.barplot(data=counts, x=counts.index.tolist(), y='Percentage', palette = current_palette)
plt.xticks(rotation=90)


In [ ]:
# Matching those with the UMAP colors
bc.pl.update_qualitative_palette( adata, palette=current_palette, group = 'leiden', checkColors=True)
#plot markers documented in Seurat tutorial to label celltypes
sc.pl.umap(adata=adata, color = ['leiden'], edgecolor = 'none')

In [ ]:
adata

In [ ]:
from random import choices
adata.obs["storage_condition"] = choices(['condition1', 'condition2'], k=len(adata.obs))
adata.obs["storage_condition"] = adata.obs["storage_condition"].astype("category")

In [ ]:
# compare counts between two 'conditions' and output as a classic dataframe
(DF) = bc.tl.count_occurrence_subset_conditions(adata=adata,
                                                                subset_variable= 'storage_condition',
                                                                condition_identifier='storage_condition',
                                                                count_variable='leiden',
                                                                return_percentage=True)
display(DF)

In [ ]:
adata.obs.head()

In [ ]:
bc.pl.celllabel_quant_boxplot(adata, subset_variable='storage_condition', count_variable = 'leiden', condition_identifier='storage_condition')#, save_fig = False);

# Split gene expression plots

In [ ]:
adata = bc.datasets.pbmc3k_filtered()
# Split gene expression plots can only handle two conditions.
adata.obs["storage_condition"] = choices(['condition1', 'condition2'], k=len(adata.obs))
adata.obs["storage_condition"] = adata.obs["storage_condition"].astype("category")
adata_subset = adata

In [ ]:
bc.pl.gene_expr_split(adata_subset, genes = ['RPS26'], split_variable= 'storage_condition')

In [ ]:
bc.pl.gene_expr_split(adata_subset, genes = ['RPS26', 'RPS4X'], split_variable= 'storage_condition')#, #group_variable = 'condition', 
                      #split_variable='condition')

In [ ]:
bc.pl.gene_expr_split(adata_subset,
                      genes = ['RPS26', 'RPS4X'], 
                      label_split_variable= 'condition', 
                     split_variable= 'storage_condition')

# Helper function:

## Conversion

From symbol to ensembl and vice-versa.

In [ ]:
bc.convert_symbol_to_ensembl( ['KRAS', 'MAP4K1'])

In [ ]:
bc.convert_ensembl_to_symbol([ 'ENSG00000104814', 'ENSG00000282928' ])